In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import napari
from FoodSeg103_data_handler import *

## Prediction

Load the images as batches

In [ ]:
img_nums = [0] #[n for n in range(0, 4900, 500)] #2750 #1234 #2314
imgs = load_food_batch(img_nums, load_gts=False)[0]
num_imgs = len(imgs)
print(f"Total images: {num_imgs}")

Check and filter for resolution

In [ ]:
resol = {im_num: np.prod(im.shape[:2]) for im_num, im in imgs.items()}

# Check for images with resolution over a certain threshold
thresh = 250_000
print("Resolutions:")
for im_num, r in resol.items(): print(f"{im_num:4d}: {r:9,d} pixels{' (!)' if r > thresh else ''}")

In [ ]:
# Filter out those images
imgs = {im_num: im for im_num, im in imgs.items() if resol[im_num] <= thresh}
num_imgs = len(imgs)
print(f"Images with resolution under {thresh:,d}: {num_imgs}")
resol_new = {im_num: np.prod(im.shape[:2]) for im_num, im in imgs.items()}
for im_num, r in resol_new.items(): print(f"{im_num:4d}: {r:7,d} pixels")

Define prediction parameters

In [ ]:
# Which scribbles to use
bins = [0.05] #[0.05, 0.3, 1.5]
suff = ["TEST"] #standing for width 3
scribble_width = 3

# Which prediction models to use
convpaint_layers = [[0]] #[[0], [0,2]]
convpaint_scalings = [[1,2]] #[[1], [1,2], [1,2,4], [1,2,4,8]]
use_dino = True
use_ilastik = True
pred_seed = 123

# Where to find and save the data
folder_path = "./FoodSeg103_results/data/food_run02"

save_res = True
show_res = True

Loop and predict

In [ ]:
for bin in bins:
    for img_num, img in imgs.items():
        for s in suff:
            print(f"IMG {img_num}: bin {bin}, suff {s}")
            for layers in convpaint_layers:
                for scalings in convpaint_scalings:
                    print(f"   ConvPaint: layers {layers}, scalings {scalings}")
                    pred_conv = pred_food_convpaint(img, folder_path, img_num, mode="all", bin=bin, scribble_width=scribble_width, suff=s, 
                                                    layer_list=layers, scalings=scalings, model="vgg16",
                                                    random_state=pred_seed,
                                                    save_res=save_res, show_res=show_res, ground_truth=None)
            if use_ilastik:
                print("   Ilastik")
                pred_ila = pred_food_ilastik(img, folder_path, img_num, mode="all", bin=bin, scribble_width=scribble_width, suff=s,
                                             random_state=pred_seed,
                                             save_res=save_res, show_res=show_res, ground_truth=None)
            if use_dino:
                print("   DINO")
                pred_dino = pred_food_dino(img, folder_path, img_num, mode="all", bin=bin, scribble_width=scribble_width, suff=s, 
                                              dinov2_model='s', dinov2_layers=(), dinov2_scales=(), upscale_order=1,
                                              random_state=pred_seed,
                                              save_res=save_res, show_res=show_res, ground_truth=None)
            print("\n")

Show the last image with ground truth, scribbles and prediction

In [ ]:
np.sum(pred_conv == load_food_data(img_num, load_image=False)[1]) / np.prod(pred_conv.shape)

In [ ]:
v = napari.Viewer()
v.add_image(img)
# v.add_labels(ground_truth)
# v.add_labels(scribbles)
v.add_labels(pred_conv)
v.add_labels(pred_ila)
v.add_labels(pred_dino)

In [ ]:
# load and add the ground truth
v.add_labels(load_food_data(img_num, load_image=False)[1])